In [ ]:
from pycocotools import _mask as mask_util
import torch

size = (100, 100)
counts = [3, 0, 2, 3, 1, 5, 1, 6, 1, 7, 1, 8, 2, 9, 1, 11, 1, 12, 1, 13, 1, 14, 1, 15, 2, 16, 2, 18]

# Calculate the number of pixels in the binary mask
num_pixels = size[0] * size[1]

# Create a binary mask tensor of zeros
mask_tensor = torch.zeros(num_pixels, dtype=torch.uint8)

# Extract the run-lengths and starting indices from the RLE
rle_counts = torch.tensor(counts[::2], dtype=torch.long)
rle_indices = torch.tensor(counts[1::2], dtype=torch.long)

# Calculate the ending indices of each run of 1s
rle_ends = rle_indices + rle_counts

# Create a list of the indices of the 1s in the mask
indices_list = [torch.arange(start, end) for start, end in zip(rle_indices, rle_ends)]

# Set the corresponding pixels in the mask to 1 using vectorized indexing
mask_tensor[torch.cat(indices_list)] = 1

# Reshape the 1D tensor into a 2D binary mask tensor
mask_tensor = mask_tensor.reshape(size[0], size[1])

mask_tensor

In [ ]:
x = bytes([17, 24, 121, 1, 12, 222, 34, 76])
print(x)
print(list(x))

In [ ]:
a = b"0550000000b1"


def encode_int_list(int_list):
    # Initialize an empty string to store the encoded data
    encoded_str = ""
    # Loop over each integer in the list
    for i in int_list:
        # Convert the integer to a binary string, removing the '0b' prefix
        binary_str = bin(i)[2:]
        # Pad the binary string with zeros until it's a multiple of 6 bits long
        padded_str = binary_str.zfill((len(binary_str) + 5) // 6 * 6)
        # Break the padded string into 6-bit chunks and convert each to an integer
        chunk_list = [int(padded_str[j : j + 6], 2) for j in range(0, len(padded_str), 6)]
        # Encode each chunk as an ASCII character in the range 48-111
        encoded_chars = [chr(chunk + 48) for chunk in chunk_list]
        # Add the encoded characters to the output string
        encoded_str += "".join(encoded_chars)
    return encoded_str


list(encode_int_list(a))

In [ ]:
a = b"0T3"
b = bytearray(a)
r = 0
for i, e in enumerate(b):
    r = r + ((e & 0x7F) << (i * 7))
if e & 0x40 != 0:
    r |= -(1 << (i * 7) + 7)
print(r)
print([int(x) for x in bin(r)[2:]])

In [ ]:
a = b"0T3"
import torch


def rleFrString(s):
    s_str = s.decode()
    s_numeric = torch.tensor([int(c) for c in s_str if c.isdigit()], dtype=torch.uint8)
    mask = s_numeric & 0x20

    positions = torch.nonzero(s_numeric, as_tuple=True)[0]
    lengths = torch.diff(positions, dim=0) - 1

    k_values = torch.arange(0, 5 * int(lengths.max()), step=5, device=s_numeric.device)
    bitshift = k_values.repeat_interleave(lengths)

    x = (s_numeric & 0x1F) << bitshift
    x = torch.cumsum(x * mask, dim=0)

    x_neg_mask = (~mask) & (s_numeric & 0x10).bool()
    x_neg = x_neg_mask.to(torch.int32) * (-1 << bitshift)
    x += torch.cumsum(x_neg, dim=0) * x_neg_mask

    cnts = torch.zeros_like(x, dtype=torch.uint32)
    cnts[2:] = x[:-2]

    return cnts


list(rleFrString(a))

In [ ]:
a.decode("utf-8")

In [ ]:
list(a.decode())

In [ ]:
a.decode("utf-8")

In [ ]:
a.decode("ascii")

In [ ]:
from pycocotools import mask as coco_mask

# Example COCO run-length encoded mask
rle = {
    "counts": bytes([3, 0, 2, 3, 1, 5, 1, 6, 1, 7, 1, 8, 2, 9, 1, 11, 1, 12, 1, 13, 1, 14, 1, 15, 2, 16, 2, 18]),
    "size": [4, 5],
}

# Decode COCO RLE mask to binary mask numpy array
mask = coco_mask.decode(rle)

In [ ]:
a = b"0550000000b1"
import numpy as np
import struct


def decode_run_length(encoded_string):
    binary_data = bytearray.fromhex(encoded_string.decode("ascii"))
    binary_string = ""
    for byte in binary_data:
        bits = bin(byte)[2:].zfill(8)
        binary_string += bits[2:]
    matrix = np.array(list(binary_string), dtype=int)
    return matrix


encoded_string = a

decoded_matrix = decode_run_length(encoded_string)
print(decoded_matrix)

In [ ]:
def rleFromString(s):
    m = len(s)
    p = 0
    k = 0
    cnts = []

    # Allocate memory for storing counts
    cnts = [0] * m
    m = 0

    # Process the input string 's'
    while s[p]:
        x = 0
        k = 0
        more = True

        # Read and decode the characters in 's' until 'more' is False
        while more:
            c = ord(s[p]) - 48
            x |= (c & 0x1F) << (5 * k)
            more = bool(c & 0x20)
            p += 1
            k += 1

            # If 'more' is False and the fifth bit (0x10) of 'c' is set, it means
            # the value of 'x' should be a negative number, so we perform sign extension
            if not more and (c & 0x10):
                x |= -1 << (5 * k)

        # Add the previous count to 'x' if there were more than 2 counts
        if m > 2:
            x += cnts[m - 2]

        # Store the decoded value of 'x' in 'cnts'
        cnts[m] = x
        m += 1
    return cnts


rleFromString(a)

In [ ]:
def rleFrBytes(s):
    m = len(s)
    cnts = [0] * m
    p = 0
    m = 0

    while p < len(s):
        x = 0
        k = 0
        more = True

        while more:
            c = s[p] - 48
            x |= (c & 0x1F) << (5 * k)
            more = bool(c & 0x20)
            p += 1
            k += 1
            if not more and (c & 0x10):
                x |= -1 << (5 * k)

        if m > 2:
            x += cnts[m - 2]

        cnts[m] = x
        m += 1

    if len(cnts) % 2 != 0:
        cnts.append(0)

    return cnts


a = b"0T3"
rleFrBytes(a)